In [2]:
## Import

# Commented out IPython magic to ensure Python compatibility.
from numpy import load
#--Data
import pandas as pd
import seaborn as sns ###
import matplotlib.pyplot as plt
import numpy as np
!pip install ftfy
import ftfy
import nltk
import json
import re
import pandas as pd
import csv
import numpy as np
import nltk
import seaborn as sb
import warnings

warnings.filterwarnings('ignore')
np.random.seed(0)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# detect and init the TPU
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [5]:
#data
train=pd.read_csv('../input/original-liar-dataset/train.csv')#, sep='\t', header=0)
test=pd.read_csv('../input/original-liar-dataset/test.csv')#, sep='\t')
val=pd.read_csv('../input/original-liar-dataset/val.csv')#, sep='\t')

#0 fake 1 true 
Original_labels={5:1, 4:1, 3:1, 1:0,2:0,0:0}
train['label'] = train['label'].map(Original_labels)
test['label'] = test['label'].map(Original_labels)
val['label'] = val['label'].map(Original_labels)

y_train=train['label']
y_val=val['label']
y_test=test['label']
print("X and y imported and Y trated")

In [7]:
"""## Embedding"""
from numpy import genfromtxt
bert_X_train = genfromtxt('../input/bert-dataset/bert_word_train_features.csv', delimiter=',')
bert_X_val = genfromtxt('../input/bert-dataset/bert_word_val_features.csv', delimiter=',')
bert_X_test = genfromtxt('../input/bert-dataset/bert_word_test_features.csv', delimiter=',')
print("X and y ok")

classe = 'label'
df_pred_val = pd.DataFrame()
df_pred_test = pd.DataFrame()
df_prob_val = pd.DataFrame()# inicializando proba
df_prob_test = pd.DataFrame()
      
df_pred_val = pd.concat([df_pred_val, val['label']], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, test['label']], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, val['label']], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, test['label']], axis=1, sort=False)
print("Dataframes created")

# classifiers
svm_bert = SVC(random_state=42, kernel='rbf', gamma=0.1, probability=True) #ver gaussiano
lr_bert = LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1')
rf_bert = RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1)
nb_bert = BernoulliNB(alpha=0.1, fit_prior=True)
mlp_bert = MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='logistic', solver='adam')
extra_bert = ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1)
knn_bert = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
print("Classifiers created")

#reshape train, val, teste to X and y 
BXtrain=[]
for feature in bert_X_train:
  feature=np.mean(feature)
  BXtrain.append(feature)
BXtrain=np.array(BXtrain)
Bytrain=np.array(y_train)

BXval=[]
for feature in bert_X_val:
  feature=np.mean(feature)
  BXval.append(feature)
BXval=np.array(BXval)
Byval=np.array(y_val)

BXtest=[]
for feature in bert_X_test:
  feature=np.mean(feature)
  BXtest.append(feature)
BXtest=np.array(BXtest)
Bytest=np.array(y_test)

print(BXtrain.shape, type(BXtrain), Bytrain.shape, type(Bytrain))
print("X bert trated")


In [8]:
# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    svm_bert = SVC(random_state=42, kernel='rbf', gamma=0.1, probability=True) #ver gaussiano
    
# train model normally
svm_bert.fit(BXtrain.reshape(-1, 1),Bytrain)

In [9]:
df_pred_val["SVM-BERT"] = svm_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval)):
  result=svm_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]
df_prob_val["SVM-BERT-0"]=list_0
df_prob_val["SVM-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")
print("Svm val ok")

In [19]:
df_pred_test["SVM-BERT"] = svm_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=svm_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]
df_prob_test["SVM-BERT-0"]=list_0
df_prob_test["SVM-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")
print("Svm test ok")

In [21]:
#LR
lr_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["LR-BERT"] = lr_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval.reshape(-1,1))):
  result=lr_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["LR-BERT-0"]=list_0
df_prob_val["LR-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#LR
df_pred_test["LR-BERT"] = lr_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=lr_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["LR-BERT-0"]=list_0
df_prob_test["LR-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")

In [25]:
#RF
rf_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["RF-BERT"] = rf_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval.reshape(-1,1))):
  result=rf_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["RF-BERT-0"]=list_0
df_prob_val["RF-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#RF
df_pred_test["RF-BERT"] = rf_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=rf_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["RF-BERT-0"]=list_0
df_prob_test["RF-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")

In [27]:
#NB
nb_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["NB-BERT"] = nb_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval.reshape(-1,1))):
  result=nb_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["NB-BERT-0"]=list_0
df_prob_val["NB-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#NB
df_pred_test["NB-BERT"] = nb_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=nb_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["NB-BERT-0"]=list_0
df_prob_test["NB-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")

In [28]:
#MLP
mlp_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["MLP-BERT"] = nb_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval.reshape(-1,1))):
  result=mlp_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["MLP-BERT-0"]=list_0
df_prob_val["MLP-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#MLP
df_pred_test["MLP-BERT"] = nb_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=mlp_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["MLP-BERT-0"]=list_0
df_prob_test["MLP-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")


In [29]:
#EXTRA
extra_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["EXTRA-BERT"] = extra_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval.reshape(-1,1))):
  result=extra_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["EXTRA-BERT-0"]=list_0
df_prob_val["EXTRA-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#EXTRA
df_pred_test["EXTRA-BERT"] = extra_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=extra_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["EXTRA-BERT-0"]=list_0
df_prob_test["EXTRA-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")

In [30]:
#KNN
knn_bert.fit(BXtrain.reshape(-1,1),Bytrain)
df_pred_val["KNN-BERT"] = knn_bert.predict(BXval.reshape(-1,1))
df_pred_val.to_csv("pred_val.csv")
list_0=[]
list_1=[]
for i in range(len(BXval)):
  result=knn_bert.predict_proba(BXval.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_val["KNN-BERT-0"]=list_0
df_prob_val["KNN-BERT-1"]=list_1
df_prob_val.to_csv("prob_val.csv")

#KNN
df_pred_test["KNN-BERT"] = knn_bert.predict(BXtest.reshape(-1,1))
df_pred_test.to_csv("pred_test.csv")
list_0=[]
list_1=[]
for i in range(len(BXtest.reshape(-1,1))):
  result=knn_bert.predict_proba(BXtest.reshape(-1,1))
  list_0=result[i][0]
  list_1=result[i][-1]

df_prob_test["KNN-BERT-0"]=list_0
df_prob_test["KNN-BERT-1"]=list_1
df_prob_test.to_csv("prob_test.csv")



In [31]:
val_pred_view=pd.read_csv('./pred_val.csv')
val_proba_view=pd.read_csv('./prob_val.csv')
test_pred_view=pd.read_csv('./pred_test.csv')
test_proba_view=pd.read_csv('./prob_test.csv')

In [32]:
val_pred_view

In [33]:
test_pred_view

In [34]:
val_proba_view

In [35]:
test_proba_view